## Regressão Logistica

In [ ]:
import keras 
import pandas as pd
import tensorflow as tf 
import seaborn as sea
import matplotlib.pyplot as plt
from  pyprind.progbar import ProgBar

: 

In [ ]:
class modelo(tf.Module):
    def __init__(self, name=None, n_features=1, seed=42):
        super().__init__(name)

        # Inicializando peso aleatóriamente
        tf.random.set_seed(seed=seed)


        rnd = tf.random.normal(shape=[1, n_features], dtype=tf.float32)
    
        ## Adicionando uma coluna de 1 = interceptor    
        params = tf.concat([
            tf.ones_like(rnd[:, 0:1]),
            rnd
        ], axis=1)
        
        self.params = tf.Variable(initial_value=params, name='params', dtype=tf.float32)

        
    def __call__(self, X):
        X_ = tf.concat([tf.ones_like(X[:, 0:1]), X], axis=1)
        return tf.matmul(X_, self.params, transpose_b=True)

    def summary(self):
        print(f'Params: {self.params.numpy().tolist()}')

### Classificação

1. **Classificação Binária :** Somente duas classes
2. **Classificação Multiclasse :** Mais de duas classes 

In [ ]:
iris = pd.read_csv('datasets/Iris.csv')
iris.head()

In [ ]:
iris.drop(columns=['Id'], inplace=True)

### Classificação Binária

In [ ]:
iris['class'] = iris['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})

In [ ]:
subset = iris.loc[iris['class'].isin([0, 1])].iloc[:,[0, 1, 4, 5]]
subset.head()

In [ ]:
sea.scatterplot(x='SepalLengthCm', y='SepalWidthCm', hue='Species', data=subset)
plt.show()

A

Função Sigmoid

$$\sigma(z) = \frac{1}{1+exp(-z)}$$

In [ ]:
def sigmoid(z):
    return 1 / (1 + tf.exp(-z))

In [ ]:
plt.plot(sigmoid(tf.linspace(-10, 10, 100)))

In [ ]:
def gradiente(x, y, name_model='modelo', epochs=1000, lr=0.01):
    model = modelo(name=name_model, n_features=x.shape[1])
    N = x.shape[0]
    
    x  = tf.Variable(x, dtype=tf.float32)
    #
    x_ = tf.concat([tf.ones_like(x[:, 0:1]), x], axis=1)
    y  = tf.Variable(y, dtype=tf.float32)


    for epoch in range(epochs):
        pred = tf.matmul(x_, a.params, transpose_b=True)

        pred = sigmoid(pred)

        grad = lr * (1/N) * tf.matmul(x_, y - pred, transpose_a=True)

        grad = tf.reshape(grad, [1, -1])

        model.params.assign_add(grad)

    return model

In [ ]:
a.params.assign_add

In [ ]:
X =  iris[['SepalLengthCm', 'SepalWidthCm']].values.astype('float32')
y =  iris['class'].values.reshape(-1, 1).astype('float32')

In [ ]:
model_result = gradiente(x=X, y=y, name_model='logistic', epochs=1000, lr=0.01) 